In [1]:
# Importing the libraries
import pandas as pd
import numpy as np
import nltk
from textblob import TextBlob

Applying the TextBlob funtion to the story

In [2]:
blob = TextBlob("यह है मोटा राजा।  मोटे राजा का है दुबला कुत्ता। मोटा राजा व दुबला कुत्ता घूमने निकले।  दुबले कुत्ते ने चिड़िया देखी। वह उसके पीछे भागा।  मोटा राजा दुबले कुत्ते के पीछे भागा।  दोनों भागे। और कई दिनों तक भागते रहे!  मोटे राजा ने दुबले कुत्ते को पकड़ ही लिया।  मोटा राजा अब दुबला है!")

Now, We shall see the sentences in the story

In [3]:
blob.sentences

[Sentence("यह है मोटा राजा।  मोटे राजा का है दुबला कुत्ता। मोटा राजा व दुबला कुत्ता घूमने निकले।  दुबले कुत्ते ने चिड़िया देखी। वह उसके पीछे भागा।  मोटा राजा दुबले कुत्ते के पीछे भागा।  दोनों भागे। और कई दिनों तक भागते रहे!"),
 Sentence("मोटे राजा ने दुबले कुत्ते को पकड़ ही लिया।  मोटा राजा अब दुबला है!")]

Now, We shall see the  words in the story

In [4]:
for words in blob.words:
    print(words)

यह
है
मोटा
राजा।
मोटे
राजा
का
है
दुबला
कुत्ता।
मोटा
राजा
व
दुबला
कुत्ता
घूमने
निकले।
दुबले
कुत्ते
ने
चिड़िया
देखी।
वह
उसके
पीछे
भागा।
मोटा
राजा
दुबले
कुत्ते
के
पीछे
भागा।
दोनों
भागे।
और
कई
दिनों
तक
भागते
रहे
मोटे
राजा
ने
दुबले
कुत्ते
को
पकड़
ही
लिया।
मोटा
राजा
अब
दुबला
है


let us find the number of sentences in the story

In [5]:
p = len(blob.sentences)
print(p)

2


Let us find the number of words in the story

In [6]:
q = (len(blob.words))
print(q)

55


In [7]:
import re

let us define the unicodes for the vowels, consonants, vowel signs etc., of the hindi language in order to define the function 
to find the syllables in the story

In [8]:
vowels = '\u0904-\u0914\u0960-\u0961\u0972-\u0977'
consonants = '\u0915-\u0939\u0958-\u095F\u0978-\u097C\u097E-\u097F'
glottal = '\u097D'

vowel_signs = '\u093E-\u094C\u093A-\u093B\u094E-\u094F\u0955-\u0957\u1CF8-\u1CF9'
nasals = '\u0900-\u0902\u1CF2-\u1CF6'
visarga = '\u0903'
nukta = '\u093C'
avagraha = '\u093D'
virama = '\u094D'

vedic_signs = '\u0951-\u0952\u1CD0-\u1CE1\u1CED'
visarga_modifiers = '\u1CE2-\u1CE8'
combining = '\uA8E0-\uA8F1'

om = '\u0950'

accents = '\u0953-\u0954'
dandas = '\u0964-\u0965'
digits = '\u0966-\u096F'
abbreviation = '\u0970'
spacing = '\u0971'

vedic_nasals = '\uA8F2-\uA8F7\u1CE9-\u1CEC\u1CEE-\u1CF1'
fillers = '\uA8F8-\uA8F9'
caret = '\uA8FA'
headstroke = '\uA8FB'

space = '\u0020'
joiners = '\u200C-\u200D'

defining the functions to find the syllables in the story

In [9]:
def syllabify(inputtext):

    syllables = []
    curr = ''

    # iterate over each character in the input. if a char belongs to a 
    # class that can be part of a syllable, then add it to the curr 
    # buffer. otherwise, output it to syllables[] right away.

    for char in inputtext:

        if re.match('[' + vowels + avagraha + glottal + om + ']', char):

            # need to handle non-initial independent vowel letters,
            # avagraha, and om

            if curr != '':
                syllables.append(curr)
                curr = char
            else:
                curr = curr + char

        elif re.match('[' + consonants + ']', char):

            # if last in curr is not virama, output curr as syllable
            # else add present consonant to curr

            if len(curr) > 0 and curr[-1] != virama:
                syllables.append(curr)
                curr = char
            else:
                curr = curr + char

        elif re.match('[' + vowel_signs + visarga + vedic_signs + ']', char):
            curr = curr + char

        elif re.match('[' + visarga_modifiers + ']', char):

            if len(curr) > 0 and curr[-1] == visarga:
                curr = curr + char
                syllables.append(curr)
                curr = ''
            else:
                syllables.append(curr)
                curr = ''

        elif re.match('[' + nasals + vedic_nasals + ']', char):

            # if last in curr is a vowel sign, output curr as syllable
            # else add present vowel modifier to curr and output as syllable

            vowelsign = re.match('[' + vowel_signs + ']$', curr)
            if vowelsign:
                syllables.append(curr)
                curr = ''
            else:
                curr = curr + char
                syllables.append(curr)
                curr = ''

        elif re.match('[' + nukta + ']', char):
            curr = curr + char

        elif re.match('[' + virama + ']', char):
            curr = curr + char

        elif re.match('[' + digits + ']', char):
            curr = curr + char

        elif re.match('[' + fillers + headstroke + ']', char):
            syllables.append(char)

        elif re.match('[' + joiners + ']', char):
            curr = curr + char

        else:
            pass
            #print ("unhandled: " + char + " ", char.encode('unicode_escape'))

    # handle remaining curr
    if curr != '':
        syllables.append(curr)
        curr = ''

    # return each syllable as item in a list
    return syllables

In [10]:
def getSyllables(inputtext):

    word_syllables = []
    all_words = []

    for word in inputtext.split():

        word = word.strip()
        word = re.sub('[\s\n\u0964\u0965\.]', '', word)

        word_syllables = syllabify(word)
        #number_syllables = len(word_syllables)

        #joined_syllables = '\u00B7'.join(word_syllables)
        joined_syllables = word_syllables

        # make list of lists containing each word
        #all_words.append([word, joined_syllables, number_syllables])
        all_words.append([word, joined_syllables])

    return all_words

In [11]:
def getSyllableStats(inputtext):

    syllcount = {}
    wordcount = {}
    word = ''
    syllable = ''
    count = ''
    syllablestatus = {}

    words = getSyllables(inputtext)

    for entry in words:

        word = entry[0]
        syllables = entry[1]
        #count = entry[2]

        #word_syllables = syllables.split('\u00B7')
        word_syllables = syllables

        # count all words in input
        if word in wordcount:
            wordcount[word] += 1
        else:
            wordcount[word] = 1

        # count all syllables in input
        for syll in word_syllables:
            if syll in syllcount:
                syllcount[syll] += 1
            else:
                syllcount[syll] = 1

    syllablestatus.update({'words' : wordcount})
    syllablestatus.update({'syllables' : syllcount})

    return (syllablestatus)

applying the functions on the story 

In [12]:
syllabify(blob)

['य',
 'ह',
 'है',
 'मो',
 'टा',
 'रा',
 'जा',
 'मो',
 'टे',
 'रा',
 'जा',
 'का',
 'है',
 'दु',
 'ब',
 'ला',
 'कु',
 'त्ता',
 'मो',
 'टा',
 'रा',
 'जा',
 'व',
 'दु',
 'ब',
 'ला',
 'कु',
 'त्ता',
 'घू',
 'म',
 'ने',
 'नि',
 'क',
 'ले',
 'दु',
 'ब',
 'ले',
 'कु',
 'त्ते',
 'ने',
 'चि',
 'ड़ि',
 'या',
 'दे',
 'खी',
 'व',
 'ह',
 'उ',
 'स',
 'के',
 'पी',
 'छे',
 'भा',
 'गा',
 'मो',
 'टा',
 'रा',
 'जा',
 'दु',
 'ब',
 'ले',
 'कु',
 'त्ते',
 'के',
 'पी',
 'छे',
 'भा',
 'गा',
 'दो',
 'नों',
 'भा',
 'गे',
 'औ',
 'र',
 'क',
 'ई',
 'दि',
 'नों',
 'त',
 'क',
 'भा',
 'ग',
 'ते',
 'र',
 'हे',
 'मो',
 'टे',
 'रा',
 'जा',
 'ने',
 'दु',
 'ब',
 'ले',
 'कु',
 'त्ते',
 'को',
 'प',
 'क',
 'ड़',
 'ही',
 'लि',
 'या',
 'मो',
 'टा',
 'रा',
 'जा',
 'अ',
 'ब',
 'दु',
 'ब',
 'ला',
 'है']

In [13]:
results = getSyllables(blob)

for x in results:
    print (x[0], ":", '\u00B7'.join(x[1]), ":", len(x[1]))

यह : य·ह : 2
है : है : 1
मोटा : मो·टा : 2
राजा : रा·जा : 2
मोटे : मो·टे : 2
राजा : रा·जा : 2
का : का : 1
है : है : 1
दुबला : दु·ब·ला : 3
कुत्ता : कु·त्ता : 2
मोटा : मो·टा : 2
राजा : रा·जा : 2
व : व : 1
दुबला : दु·ब·ला : 3
कुत्ता : कु·त्ता : 2
घूमने : घू·म·ने : 3
निकले : नि·क·ले : 3
दुबले : दु·ब·ले : 3
कुत्ते : कु·त्ते : 2
ने : ने : 1
चिड़िया : चि·ड़ि·या : 3
देखी : दे·खी : 2
वह : व·ह : 2
उसके : उ·स·के : 3
पीछे : पी·छे : 2
भागा : भा·गा : 2
मोटा : मो·टा : 2
राजा : रा·जा : 2
दुबले : दु·ब·ले : 3
कुत्ते : कु·त्ते : 2
के : के : 1
पीछे : पी·छे : 2
भागा : भा·गा : 2
दोनों : दो·नों : 2
भागे : भा·गे : 2
और : औ·र : 2
कई : क·ई : 2
दिनों : दि·नों : 2
तक : त·क : 2
भागते : भा·ग·ते : 3
रहे! : र·हे : 2
मोटे : मो·टे : 2
राजा : रा·जा : 2
ने : ने : 1
दुबले : दु·ब·ले : 3
कुत्ते : कु·त्ते : 2
को : को : 1
पकड़ : प·क·ड़ : 3
ही : ही : 1
लिया : लि·या : 2
मोटा : मो·टा : 2
राजा : रा·जा : 2
अब : अ·ब : 2
दुबला : दु·ब·ला : 3
है! : है : 1


In [14]:
from IPython.display import HTML, display
import tabulate
display(HTML(tabulate.tabulate(results, tablefmt='html')))

यह,"['य', 'ह']"
है,['है']
मोटा,"['मो', 'टा']"
राजा,"['रा', 'जा']"
मोटे,"['मो', 'टे']"
राजा,"['रा', 'जा']"
का,['का']
है,['है']
दुबला,"['दु', 'ब', 'ला']"
कुत्ता,"['कु', 'त्ता']"
मोटा,"['मो', 'टा']"


In [15]:
stats = getSyllableStats(blob)
for k, v in stats.items():
    print (k)
    for x, y in v.items():
        print ("\t", x, ":", y)
    print ("total", k, "=", sum(v.values()))
    print ("unique", k, "=", len(v), end='\n\n')

words
	 यह : 1
	 है : 2
	 मोटा : 4
	 राजा : 6
	 मोटे : 2
	 का : 1
	 दुबला : 3
	 कुत्ता : 2
	 व : 1
	 घूमने : 1
	 निकले : 1
	 दुबले : 3
	 कुत्ते : 3
	 ने : 2
	 चिड़िया : 1
	 देखी : 1
	 वह : 1
	 उसके : 1
	 पीछे : 2
	 भागा : 2
	 के : 1
	 दोनों : 1
	 भागे : 1
	 और : 1
	 कई : 1
	 दिनों : 1
	 तक : 1
	 भागते : 1
	 रहे! : 1
	 को : 1
	 पकड़ : 1
	 ही : 1
	 लिया : 1
	 अब : 1
	 है! : 1
total words = 55
unique words = 35

syllables
	 य : 1
	 ह : 2
	 है : 3
	 मो : 6
	 टा : 4
	 रा : 6
	 जा : 6
	 टे : 2
	 का : 1
	 दु : 6
	 ब : 7
	 ला : 3
	 कु : 5
	 त्ता : 2
	 व : 2
	 घू : 1
	 म : 1
	 ने : 3
	 नि : 1
	 क : 4
	 ले : 4
	 त्ते : 3
	 चि : 1
	 ड़ि : 1
	 या : 2
	 दे : 1
	 खी : 1
	 उ : 1
	 स : 1
	 के : 2
	 पी : 2
	 छे : 2
	 भा : 4
	 गा : 2
	 दो : 1
	 नों : 2
	 गे : 1
	 औ : 1
	 र : 2
	 ई : 1
	 दि : 1
	 त : 1
	 ग : 1
	 ते : 1
	 हे : 1
	 को : 1
	 प : 1
	 ड़ : 1
	 ही : 1
	 लि : 1
	 अ : 1
total syllables = 112
unique syllables = 51



from this we observe that, there are 112 syllables and 55 words

 to check the readablity we will use the Flesch Reading Ease formula,

Flesch Reading Ease = 206.835-(1.015xASL)-(84.5xASW)

and Flesch-kincaid Grade-Level = (0.39xASL)+(11.8XASW)-15.59

Where, ASL( = Total Number of words/Total number of sentences

       ASW = Total syllable count/number of words

In [16]:
def ASL (w,s):
    ASL = w/s
    return ASL

In [17]:
def ASW(sy,w):
    asw = sy/w
    return asw

In [18]:
def FRE(ASL,ASW):
    flesch_reading_ease = 206.835-(1.015 * ASL)-(84.5 * ASW)
    return flesch_reading_ease

In [19]:
def FKGL(ASL,ASW):
    flesch_kincaid_Grade_level = (0.39*ASL)+(11.8*ASW)-15.59
    return flesch_kincaid_Grade_level

In [20]:
ASL(55,2)

27.5

In [21]:
ASW(112,55)

2.036363636363636

In [22]:
FRE(27.5,2.03636364)

6.849772420000022

In [23]:
FKGL(27.5,2.03636364)

19.164090952

Here,we observed that Flesch reading ease score lies between 0-29 and the Flesch kincaid grade level is 19.164.
Then, the story is very confusing and can be read by the people who are graduated.